In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.core.sql import SQLExecutor2

# Read recipe inputs
pda_MV_PROJECT_MASTER = dataiku.Dataset("PDA_MV_PROJECT_MASTER")
pda_MV_PROJECT_MASTER_df = pda_MV_PROJECT_MASTER.get_dataframe()
pda_STG_ENRICH_PROJECT_METADATA_BASE_QUERY = dataiku.Dataset("PDA_STG_ENRICH_PROJECT_METADATA_BASE_QUERY")
pda_STG_ENRICH_PROJECT_METADATA_BASE_QUERY_df = pda_STG_ENRICH_PROJECT_METADATA_BASE_QUERY.get_dataframe()

PDA_TEST_PROJECT_METADATA = dataiku.Dataset("PDA_TEST_PROJECT_METADATA")
PDA_TEST_PROJECT_METADATA_df = PDA_TEST_PROJECT_METADATA.get_dataframe()
# PDA_TEST_PROJECT_METADATA_df

In [0]:
pda_MV_PROJECT_MASTER_df

In [0]:
client = dataiku.api_client()
DSSProject = client.get_project("PDANEXUSANALYTICS")
variables = DSSProject.get_variables()
variables["standard"]["Project_Metadata_New_Columns"]= ''
DSSProject.set_variables(variables)

In [0]:
# merge enrich data with project master data
PDA_MV_PROJECT_METADATA_df = pd.merge(pda_STG_ENRICH_PROJECT_METADATA_BASE_QUERY_df,pda_MV_PROJECT_MASTER_df, on='CANDIDATE_CODE',  how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
PDA_MV_PROJECT_METADATA_df['Dummy111'] ='Dummy'
PDA_MV_PROJECT_METADATA_df['Dummy114'] ='Dummy'
PDA_MV_PROJECT_METADATA_df['Dummy11'] ='Dummy'
PDA_MV_PROJECT_METADATA_df['Dummy113'] ='Dummy'
PDA_MV_PROJECT_METADATA_df['Dummy1131'] ='Dummy'

In [0]:
PDA_MV_PROJECT_METADATA_df['CANDIDATE_PRIMARY_FORMULATION']= PDA_MV_PROJECT_METADATA_df['CANDIDATE_PRIMARY_FORMULATION'].astype(float)

In [0]:
def load_to_partition(df, partition_table,partitioned_on, writeWithSchema = False):
    #Group dataframe for easy partitioning
    partition_table_dataset = dataiku.Dataset(partition_table)
    GROUPED_df = df.groupby([partitioned_on])
    try:
        for key, TMP_PORTFOLIO_df in GROUPED_df:
            partition_table_dataset.set_write_partition(key)
#             #print('Writing Portfolio: ', str(int(key)), ' Rows: ', str(len(TMP_PORTFOLIO_df.index)))
#     #         logging.info('***********Writing to ' + partition_table +': ' +  str(int(key)) + ' Rows: ' + str(len(TMP_PORTFOLIO_df.index)) )

            if writeWithSchema:
                partition_table_dataset.write_schema_from_dataframe(df)

            partition_table_dataset.write_from_dataframe(TMP_PORTFOLIO_df)
        return 'Success'
    except Exception as e:
        return 'Failed'

# Load_Status = load_to_partition(PDA_MV_PROJECT_METADATA_df, "PDA_TEST_PROJECT_METADATA",'PORTFOLIO_ID', True)

# if Load_Status == 'Success':
#     pass
# elif Load_Status == 'Failed':
#     print('fail')

In [0]:
def gen_load_alter_cols(df):
    # print(PDA_MV_PROJECT_METADATA_df.dtypes)
    col_names = df.dtypes.keys().to_list()
    col_dtypes = list(df.dtypes.values)

    map_dtype = [("float64","double"), ("object","string"), ("int64","int")]

    new_datatype = []
    for ele in col_dtypes:
        for ele1 in map_dtype:
            if ele == ele1[0]:
                new_datatype.append(ele1[1])

    col_dtype = dict(zip(col_names,new_datatype))
    list_of_new_cols_dtypes = []
    for key, value in col_dtype.items():
        a = key + ' ' + value
        list_of_new_cols_dtypes.append(a)

    alter_table_cols = np.array(list_of_new_cols_dtypes, str)
    joined_string =    ",".join([str for str in alter_table_cols])

    sqlVar =  joined_string

    variables["standard"]["Project_Metadata_New_Columns"] = sqlVar
    DSSProject.set_variables(variables)
    return 'loaded the alter table list of columns to global variable: Project_Metadata_New_Columns'

# alter_col_list = gen_load_alter_cols(only_df)

In [0]:
def missing_cols(df1,df2):
    df1_list = [x.upper() for x in list(df1)]
    df2_list = [x.upper() for x in list(df2)]
    if df1_list == df2_list:
        print("Both Datasets are Identical")
        print(df1['CANDIDATE_PRIMARY_FORMULATION'].dtype)
        Load_Status = load_to_partition(df1, "PDA_TEST_PROJECT_METADATA",'PORTFOLIO_ID', True)
        return 'Loading data into partition Table: ' + Load_Status
    elif len(df1_list) > len(df2_list):
        print(df1_list)
        print(df2_list)
        final_list = list((set(df1_list).difference(df2_list)))
        print('Additional Columns in PDA_MV_PROJECT_METADATA_df:',final_list)
        Load_Status = gen_load_alter_cols(df1[final_list])
        return Load_Status
    elif len(df1_list) < len(df2_list):
        final_list = list((set(df2_list).difference(df1_list)))
        print('Additional Columns in PDA_TEST_PROJECT_METADATA:',final_list)
        print(df1['CANDIDATE_PRIMARY_FORMULATION'].dtype)
        for col in final_list:
            df1[col] = ''
        df1=df1[df2.columns]
        Load_Status = load_to_partition(df1, "PDA_TEST_PROJECT_METADATA",'PORTFOLIO_ID', True)
        return 'Loading data into partition Table: ' + Load_Status

In [0]:
new_cols_list = missing_cols(PDA_MV_PROJECT_METADATA_df,PDA_TEST_PROJECT_METADATA_df)
new_cols_list

In [0]:
user = dataiku.get_custom_variables()['ENRICH_PORTFOLIO_IDS_TO_INCLUDE']
user